Due to the limitation of sympy, we can not simplify Summation notation when K is a symbol. So we define K as a fixed integer

In [1]:
from sympy import *
import numpy as np

init_printing()

K = 3
i, j, k = symbols("i j k", integer=True)
L = symbols("L")
# K = symbols('K', integer=True, positive=True)
f = IndexedBase("f")
g = IndexedBase("g")
e = IndexedBase("e")
x = IndexedBase("x")
x0 = symbols("x0")
fs = symbols("f_*")
xs = symbols("x_*")
tau = symbols("tau")
b = IndexedBase("b")
R = symbols("R")
u = IndexedBase("u")

A = IndexedBase("A")
alpha = IndexedBase("alpha")

A_to_alpha = {A[-1]: 0, A[0]: 1, alpha[0]: 1, A[1]: 1 + alpha[1]}
for i in range(2, K + 1):
    A_to_alpha[A[i]] = A_to_alpha[A[i - 1]] + alpha[i]


# a_symbol = IndexedBase('alpha')
def calculate_theta(k, i):
    if i == k - 1:
        result = (2 * alpha[i] * alpha[k] + A[k - 1]) / A[k]
    elif i <= k - 2:
        result = 2 * alpha[k] * alpha[i] / A[k] + A[k - 1] / A[k] * calculate_theta(
            k - 1, i
        )
    else:
        result = 0
    return result.subs({A[0]: 1, A[-1]: 0, alpha[0]: 1})


theta = [[calculate_theta(k, i) for i in range(k)] for k in range(0, K + 1)]


def calculate_x(k):
    if k == 0:
        return x0
    else:
        return x0 - 1 / L * sum([theta[k][i] * (g[i] + e[i]) for i in range(k)])


x_expr = {x[k]: calculate_x(k) for k in range(K + 1)}
x_expr

⎧                                                                              ↪
⎪                                                                              ↪
⎪                                                                ⎛2⋅alpha[1] + ↪
⎨                                                                ⎜──────────── ↪
⎪                     (2⋅alpha[1] + 1)⋅(e[0] + g[0])             ⎝     A[2]    ↪
⎪x[0]: x₀, x[1]: x₀ - ──────────────────────────────, x[2]: x₀ - ───────────── ↪
⎩                                 L⋅A[1]                                       ↪

↪                                                                              ↪
↪                                                                              ↪
↪  1   2⋅alpha[2]⎞                 (A[1] + 2⋅alpha[1]⋅alpha[2])⋅(e[1] + g[1])  ↪
↪ ── + ──────────⎟⋅(e[0] + g[0]) + ──────────────────────────────────────────  ↪
↪         A[2]   ⎠                                    A[2]                     ↪
↪ ─────────────────────────

In [2]:
# initial inequality


def eq_vi(k):
    eq = (
        f[k + 1]
        - f[k]
        + g[k + 1] * (x[k] - x[k + 1])
        + 1 / (2 * L) * (g[k] - g[k + 1]) ** 2
    )
    return eq.subs(x_expr)


def eq_vs(k):
    eq = f[k] - fs + g[k] * (xs - x[k]) + 1 / (2 * L) * g[k] ** 2
    return eq.subs(x_expr)


ineq_init = Sum(A[k] / A[K] * eq_vi(k), (k, 0, K - 1)) + Sum(
    (A[k] - A[k - 1]) / A[K] * eq_vs(k), (k, 0, K) ) + tau * ( (x[0]- xs)**2 - R**2) + Sum(u[k] * ( e[k]**2 - b[k]**2), (k, 0, K - 1))


# ineq_init = ineq_init.doit().subs({A[0]: 1, A[-1]: 0}).subs(x_expr).cancel()
ineq_init

                                                             3                 ↪
                                                           _____               ↪
                              2                            ╲                   ↪
                             ___                            ╲                  ↪
                             ╲                               ╲                 ↪
  ⎛   2                2⎞     ╲   ⎛      2       2⎞           ╲  (-A[k - 1] +  ↪
τ⋅⎝- R  + (-x_* + x[0]) ⎠ +   ╱   ⎝- b[k]  + e[k] ⎠⋅u[k] +    ╱                ↪
                             ╱                               ╱   ───────────── ↪
                             ‾‾‾                            ╱                  ↪
                            k = 0                          ╱                   ↪
                                                           ‾‾‾‾‾               ↪
                                                           k = 0               ↪

↪                          

In [3]:
LHS_init = f[K] - fs - 1 / (2*L) * g[K]**2 
RHS_init = tau * (R**2 - (x[0]- xs)**2) + Sum(u[k] * (b[k]**2 - e[k]**2), (k, 0, K - 1)) - Sum(A[k] / A[K] * g[k+1]*(x[k]-x[k+1]), (k,0,K-1)) - Sum((A[k] - A[k - 1]) / A[K] * g[k] * (xs - x[k]), (k, 0, K)) - 1/(2*L) * Sum(A[k] / A[K] * (g[k] - g[k + 1])**2, (k, 0, K - 1)) - 1/(2*L) * Sum((A[k] - A[k - 1]) / A[K] * g[k]**2, (k, 0, K)) - 1 / (2*L) * g[K]**2
res = LHS_init - RHS_init - ineq_init
res.doit().subs(A[-1],0).cancel()

0

In [4]:
# replace x_k with iGOGM

# double sum part
double_sum = 0
for num_k in range(K):
    for num_i in range(num_k+1):
        double_sum +=  alpha[num_k+1]*alpha[num_i]/A[K]*g[num_k+1]*(g[num_i]+e[num_i])


RHS_iGOGM = tau * (R**2 - (x[0]- xs)**2) + Sum(u[k] * (b[k]**2 - e[k]**2), (k, 0, K - 1)) + Sum(alpha[k]/A[K]*g[k] *(x0-xs),(k,0,K)) - 1/L *Sum(A[k]/A[K]*g[k]**2,(k,0,K)) + 1/L *Sum(A[k]/A[K]*g[k]*g[k+1], (k,0,K-1)) - 2/L *double_sum - 1/L * Sum(A[k]/A[K]*g[k+1]*(g[k]+e[k]),(k,0,K-1))

res = RHS_init - RHS_iGOGM
res.doit().subs(A[-1],0).subs(x_expr).subs(A_to_alpha).cancel()

0

In [5]:
# first Shur complement
double_sum1 = 0
for num_k in range(K+1):
    for num_i in range(num_k):
        double_sum1 +=  alpha[num_k]*alpha[num_i] / (A[K]**2) *g[num_k]*g[num_i]



double_sum2 = 0

for num_k in range(K):
    for num_i in range(num_k+1):
        double_sum2 += alpha[num_k+1]*alpha[num_i]/(A[K])*g[num_k+1]*(g[num_i]+e[num_i])





RHS_shur_tau1= tau * (R**2 - (x[0]- xs)**2) + Sum(u[k] * (b[k]**2 - e[k]**2), (k, 0, K - 1)) - 1/tau *(tau*(x0-xs) - 1/2 * Sum(alpha[k]/A[K]*g[k],(k,0,K)))**2 + tau*(x0-xs)**2 + Sum((alpha[k]**2/(4*tau*A[K]**2) - A[k]/(L*A[K]))*g[k]**2,(k,0,K)) + 1/ (2*tau) * double_sum1 + 1/L * Sum(A[k]/A[K]*g[k]*g[k+1],(k,0,K-1)) - 2/L*double_sum2 - 1/L *Sum(A[k]/A[K]*g[k+1]*(g[k]+e[k]),(k,0,K-1))



res=RHS_shur_tau1 - RHS_iGOGM
res.doit().cancel()


0

In [6]:
# replace tau = L/4A_K
double_sum = 0
for num_k in range(K):
    for num_i in range(num_k+1):
        double_sum += alpha[num_k+1]*alpha[num_i]/(A[K])*g[num_k+1]*(e[num_i])

RHS_shur_tau2 = L*R**2/(4*A[K]) + Sum(u[k] * (b[k]**2 - e[k]**2), (k, 0, K - 1)) - 4*A[K]/L * (L/(4*A[K])*(x0-xs) - 1/2 * Sum(alpha[k]/A[K]*g[k],(k,0,K)))**2 - Sum((A[k]-alpha[k]**2)/(L*A[K])* g[k]**2,(k,1,K) ) - 2/L*double_sum - 1/L *Sum(A[k]/A[K]*g[k+1]*e[k],(k,0,K-1))

res = RHS_shur_tau2 - RHS_shur_tau1.subs(tau, L/(4*A[K]) )
res.doit().subs(x_expr).subs(A_to_alpha).cancel()


0

In [7]:
# Verify the second Shur complement
double_sum = 0
for num_k in range(K):
    for num_i in range(num_k+1):
        double_sum += alpha[num_k+1]*alpha[num_i]/(A[K])*g[num_k+1]*e[num_i]

LHS_shur_D = L*R**2/(4*A[K]) + Sum(u[k] * (b[k]**2 - e[k]**2), (k, 0, K - 1))  - Sum((A[k]-alpha[k]**2)/(L*A[K])* g[k]**2,(k,1,K) ) - 2/L*double_sum - 1/L *Sum(A[k]/A[K]*g[k+1]*e[k],(k,0,K-1))

double_sum1 = 0
for num_k in range(K):
    inner_sum = 0
    for num_i in range(num_k+1):
        inner_sum += alpha[num_k+1]*alpha[num_i]*e[num_i] 
    
    double_sum1 += 1/( L*A[K]*(A[num_k+1]-alpha[num_k+1]**2) ) * ( (A[num_k+1]-alpha[num_k+1]**2) * g[num_k+1] + inner_sum + 1/2 *A[num_k]*e[num_k] )**2 

double_sum2 = 0
for num_k in range(K):
    inner_sum = 0
    for num_i in range(num_k):
        inner_sum += alpha[num_k+1]**2 *alpha[num_i]**2 * e[num_i]**2 
    
    double_sum2 += 1/(L*A[K]*(A[num_k+1]-alpha[num_k+1]**2) ) * inner_sum

triple_sum1 = 0
for num_k in range(K):
    inner_sum = 0
    for num_i in range(num_k+1):
        for num_j in range(num_i):
            inner_sum += 2* alpha[num_k+1]**2 * alpha[num_i] * alpha[num_j] * e[num_i] * e[num_j]
    triple_sum1 += 1/(L*A[K]*(A[num_k+1]-alpha[num_k+1]**2) ) * inner_sum

double_sum3 =0 
for num_k in range(K):
    inner_sum = 0
    for num_i in range(num_k):
        inner_sum += alpha[num_k+1] * alpha[num_i] * A[num_k]*e[num_i]*e[num_k]
    double_sum3 += 1/(L*A[K]*(A[num_k+1]-alpha[num_k+1]**2) ) * inner_sum


RHS_shur_D = L*R**2/(4*A[K]) + Sum(u[k] * (b[k]**2 - e[k]**2), (k, 0, K - 1)) - double_sum1 + double_sum2 + triple_sum1 + double_sum3 + Sum((2*alpha[k+1]*alpha[k] + A[k])**2 / (4*L*A[K]*(A[k+1]-alpha[k+1]**2)) * e[k]**2, (k,0, K-1) )

res = LHS_shur_D - RHS_shur_D
res.doit().subs(x_expr).subs(A_to_alpha).cancel()

0

In [8]:
# change incides
RHS_shur_D1 =  double_sum2 + triple_sum1 + double_sum3 + Sum((2*alpha[k+1]*alpha[k] + A[k])**2 / (4*L*A[K]*(A[k+1]-alpha[k+1]**2)) * e[k]**2, (k,0, K-1) )


double_sum1 = 0
for num_k in range(K):
    for num_j in range(num_k+1,K):
        double_sum1 += (alpha[num_k]**2 * alpha[num_j+1]**2) / (L*A[K]*(A[num_j+1]-alpha[num_j+1]**2)) * e[num_k]**2

double_sum2 = 0
for num_k in range(K):
    for num_i in range(num_k):
        double_sum2 += alpha[num_k+1]*alpha[num_i] * (A[num_k]+2*alpha[num_k+1] * alpha[num_k])/(L* A[K]* (A[num_k+1]-alpha[num_k+1]**2))*e[num_k]*e[num_i]

triple_sum = 0
for num_k in range(K):
    for num_i in range(num_k):
        for num_j in range(num_k+1,K):
            triple_sum += 2 * alpha[num_j+1]**2 * alpha[num_k] * alpha[num_i] / (L*A[K]*(A[num_j+1]-alpha[num_j+1]**2)) * e[num_k] * e[num_i]


RHS_shur_D2 = Sum( (2*alpha[k+1]*alpha[k]+ A[k])**2 / (4*L*A[K]*(A[k+1]-alpha[k+1]**2)) * e[k]**2, (k,0, K-1) ) + double_sum1 +double_sum2 + triple_sum 

res = RHS_shur_D1 - RHS_shur_D2
res.doit().subs(x_expr).subs(A_to_alpha).cancel()


0

In [9]:
# Use P notation
P = IndexedBase("P")

def calculate_P(k,i):
    if i == k :
        return (2*alpha[k+1]*alpha[k] +A[k])**2 / (4*L*A[K]*(A[k+1]-alpha[k+1]**2)) + Sum( alpha[k]**2 * alpha[j+1]**2 / (L*A[K] * (A[j+1]-alpha[j+1]**2)), (j,k+1,K-1) )
    else:
        return alpha[k+1]*alpha[i] * (A[k]+2*alpha[k+1]*alpha[k])/(L*A[K]*(A[k+1]-alpha[k+1]**2)) + Sum(2*alpha[j+1]**2 * alpha[k] * alpha[i] / (L*A[K]*(A[j+1]-alpha[j+1]**2)), (j,k+1,K-1))


P_expr = {P[k,i]: calculate_P(k,i) for k in range(K) for i in range(k+1)}
P_expr

⎧           2                                                                  ↪
⎪         _____                                                                ↪
⎪         ╲                                                                    ↪
⎪          ╲                                                                   ↪
⎪           ╲                2             2                                   ↪
⎪            ╲       alpha[0] ⋅alpha[j + 1]         (A[0] + 2⋅alpha[0]⋅alpha[1 ↪
⎨P[0, 0]:    ╱  ───────────────────────────────── + ────────────────────────── ↪
⎪           ╱     ⎛                       2⎞             ⎛               2⎞    ↪
⎪          ╱    L⋅⎝A[j + 1] - alpha[j + 1] ⎠⋅A[3]    4⋅L⋅⎝A[1] - alpha[1] ⎠⋅A[ ↪
⎪         ╱                                                                    ↪
⎪         ‾‾‾‾‾                                                                ↪
⎪         j = 1                                                                ↪
⎩                           

In [10]:
# Check the P notation is correct
double_sum = 0
for num_k in range(K):
    for num_i in range(num_k):
        double_sum += P[num_k,num_i] * e[num_k] * e[num_i]
EQ_P = Sum(P[k,k] * e[k]**2, (k, 0, K - 1)) + double_sum

res=EQ_P.doit().subs(P_expr).doit() - RHS_shur_D2.doit()
res.cancel()

0

In [ ]:
# Check the equations for P are equal (omit the middle steps)

double_sum1 = 0
for num_k in range(K):
    for num_i in range(num_k):
        double_sum1 += 1/2* P[num_k,num_i] * (e[num_k]- e[num_i])**2
    

double_sum2 = 0
for num_k in range(K):
    inner_sum = P[num_k,num_k]
    for num_i in range(num_k):
        inner_sum +=   1/2 *P[num_k,num_i]
    for num_i in range(num_k+1,K):
        inner_sum += 1/2 * P[num_i,num_k]
    double_sum2 += inner_sum * e[num_k]**2

EQ_P_final = -double_sum1 + double_sum2 
res=EQ_P_final - EQ_P
res.doit().cancel()

0

In [13]:
# Check the u value
def calculate_u(k):

    result = A[k]*(1+2*alpha[k+1])*(A[k]+2*alpha[k]*alpha[k+1])/(4*L*A[K]*(A[k+1]-alpha[k+1]**2))
    for num_i in range(k+1,K):
        result += A[num_i]*(1+2*alpha[num_i+1])*alpha[k]*alpha[num_i+1]/(2*L*A[K]*(A[num_i+1]-alpha[num_i+1]**2))
    return result

for k in range(K):
    u_P_express = P[k,k]
    for num_i in range(0,k):
        u_P_express += 1/2 * P[k,num_i] 
    for num_i in range(k+1,K):
        u_P_express += 1/2 *P[num_i,k]
    
    res=u_P_express.subs(P_expr) - calculate_u(k)
    print('k='+str(k)+' res='+str(res.doit().subs(A_to_alpha).cancel()))


k=0 res=0
k=1 res=0
k=2 res=0
